# Implementation of RFC 4226 - HOPT Algorithm
___HMAC based One Time Password___

...in python

* [RFC 4226](https://tools.ietf.org/html/rfc4226)
* [Wikipedia](https://en.wikipedia.org/wiki/HMAC-based_One-time_Password_algorithm)

In [1]:
import hashlib
import hmac
import base64

In [2]:
# Control excessive output to console
debug = False
def dbg(data):
    if (debug):
        print(data)

In [3]:
# Prepare Counter
def get_counter(counter):
    return counter.to_bytes(8, byteorder='big')

In [4]:
### Define key, message, Block size, hashing algorithm
hash_algo = "sha1"
B = 64
counter = 1
shared_secret = b'BASE32SECRET3232'
# OTP Length
Digits = 6
# Google Authenticator Compatibility (BASE-32)
key=base64.b32decode(shared_secret)
dbg("Key Base32 Decode :")
dbg(key)

In [5]:
### Implement the HMAC Algorithm. For details see the rfc2104.ipynb at
# https://github.com/lordloh/OPT_algorithms/blob/master/rfc2104.ipynb

def my_hmac(key, message):
    trans_5C = bytes((x ^ 0x5C) for x in range(256))
    trans_36 = bytes((x ^ 0x36) for x in range(256))
    K_zpad=key.ljust(B,b'\0')    
    K_ipad=K_zpad.translate(trans_36)
    K_opad=K_zpad.translate(trans_5C)
    hash1 = hashlib.new(hash_algo, K_ipad+message).digest()
    hmac_hash = hashlib.new(hash_algo, K_opad + hash1).digest()
    return hmac_hash

In [6]:
### Dynamic Truncation
def dynamic_truncate(b_hash):
    hash_len=len(b_hash)
    int_hash = int.from_bytes(b_hash, byteorder='big')
    #str_hash = b_hash.hex()
    offset = int_hash & 0xF
    dbg ("\noffset = Lower 4 bits of hash = " + hex(int_hash & 0xF)+" = "+str(offset)+"\n")
    # Geterate a mask to get bytes from left to right of the hash
    n_shift = 8*(hash_len-offset)-32
    MASK = 0xFFFFFFFF << n_shift
    dbg ("Truncate MASK:"+hex(MASK))
    P = (int_hash & MASK)>>n_shift
    dbg ("Truncated hash (hex) : "+hex(P))
    dbg ("Truncated hash (int) : "+str(P))
    LSB_31 = P & 0x7FFFFFFF
    dbg ("Truncated hash LSB 31 bits (hex) : "+hex(LSB_31))
    dbg ("Truncated hash LSB 31 bits (int) : "+str(LSB_31))
    return LSB_31

In [7]:
def generate_HOTP(counter):
    C = get_counter(counter)
    dbg("Counter: "+str(counter)+"\nCounter Byte :")
    dbg(C)

    hmac_hash = my_hmac(key,C)
    dbg("HMAC Hash:\n" + hmac_hash.hex())

    trc_hash = dynamic_truncate(hmac_hash)
    HOTP = "{:06}".format(trc_hash % (10**Digits))
    dbg("\nHOPT : "+str(HOTP))
    dbg("----------------------")
    return HOTP

In [8]:
myHOTPs=[(generate_HOTP(x)) for x in range(10)]

In [9]:
print(myHOTPs)

['260182', '055283', '795760', '172916', '437628', '220505', '845989', '311663', '850732', '285195']


## Compare with pyOTP Implementation

In [10]:
# Python
import pyotp

In [11]:
hotp1=pyotp.HOTP(shared_secret)

In [12]:
# Generate 0..9 HOTP codes
pyHOTPs=[(hotp1.at(x)) for x in range(10)]


In [13]:
print(pyHOTPs)

['260182', '055283', '795760', '172916', '437628', '220505', '845989', '311663', '850732', '285195']
